# CCTPY CUDA 程序开发笔记

## hello-world

先用 hello-world 回顾一下利用纯 C 语言和 pychuda 开发 CUDA 程序

<pre>
#include &lt;stdio.h&gt;

// 需要从 CPU 端调用，让 GPU 执行的函数称为内核函数，需要用 __global__ 修饰
__global__ void kernel_function(){
    printf("hello, world! -- from gpu\n");
}

int main() {
    printf("hello, world! -- from cpu\n");

    // 调用内核函数，使用 <<<>>> 分配块和线程
    // 一个块内有多个线程，块内支持内存共享
    // 块和线程都是三维结构
    // 下面定义了 1*1*1 个块，1*1*1 个线程
    kernel_function<<<dim3(1,1,1),dim3(1,1,1)>>>();
}
</pre>

In [11]:
# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule

# 内核函数定义
mod = SourceModule("""
    // 需要从 CPU 端调用，让 GPU 执行的函数称为内核函数，需要用 __global__ 修饰
    __global__ void kernel_function(){
        printf("hello, world! -- from gpu\\n");
    }
""")

# 获取内核函数
kernel_function = mod.get_function("kernel_function")

# 调用内核函数
# grid 就是块数目定义，block 是块内内存
kernel_function(grid=(1,1,1), block=(1,1,1))
print("hello, pycuda! -- from python")

"""
注意：jupyter notebook 中没有打印 GPU 运行信息，可能是监听的 IO 流不对
在独立的 py 文件中，能收到 GPU 的打印
"""

hello, pycuda! -- from python


'\n注意：jupyter notebook 中没有打印 GPU 运行信息，可能是监听的 IO 流不对\n在独立的 py 文件中，能收到 GPU 的打印\n'

## 块和线程

利用 blockIdx.x 可以访问块 x 方向编号

利用 threadIdx.x 可以访问线程 x 方向编号

GPU 加速粒子跟踪，我的想法是：一个块负责一个束线的一个粒子

<pre>
#include &lt;stdio.h&gt;

// 需要从 CPU 端调用，让 GPU 执行的函数称为内核函数，需要用 __global__ 修饰
__global__ void kernel_function(){
    int bid = blockIdx.x;
    int tid = threadIdx.x;
    printf("bid=%d, tid=%d\n",bid,tid);
}

int main() {
    printf("hello, world! -- from cpu\n");

    // 调用内核函数，使用 <<<>>> 分配块和线程
    // 一个块内有多个线程，块内支持内存共享
    // 块和线程都是三维结构
    // 下面定义了 1*1*1 个块，1*1*1 个线程

    kernel_function<<<dim3(1,1,1),dim3(1,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
    kernel_function<<<dim3(3,1,1),dim3(1,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
    kernel_function<<<dim3(1,1,1),dim3(3,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
    kernel_function<<<dim3(2,1,1),dim3(2,1,1)>>>();
    cudaDeviceSynchronize();
    printf("---------\n");
}
</pre>

**程序输出**

<pre>
hello, world! -- from cpu
bid=0, tid=0
---------
bid=0, tid=0
bid=2, tid=0
bid=1, tid=0
---------
bid=0, tid=0
bid=0, tid=1
bid=0, tid=2
---------
bid=0, tid=0
bid=0, tid=1
bid=1, tid=0
bid=1, tid=1
---------
</pre>

# CUDA 简单运算

## 单个数加法


In [24]:
# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np

# 内核函数定义
mod = SourceModule("""
    __global__ void add(float *a, float *b,float *ret){
        *ret = *a + *b;
    }
""")


add = mod.get_function("add")

a = np.array([10.0], dtype=np.float32)
b = np.array([2], dtype=np.float32)
c = np.empty(1, dtype=np.float32)

add(drv.In(a), drv.In(b), drv.Out(c), grid=(1,1,1), block=(1,1,1))

print(c)

[12.]


## 向量加法

In [30]:
# -*- coding: utf-8 -*-

import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
import numpy as np

# 内核函数定义
mod = SourceModule("""
    __global__ void add(float *a, float *b,int *length,float *ret){
        int tid = threadIdx.x;
        if(tid < *length)
            ret[tid] = a[tid] + b[tid];
    }
""")


add = mod.get_function("add")

length = 5
a = np.random.rand(length).astype(np.float32)
b = np.random.rand(length).astype(np.float32)
c = np.empty_like(a)

add(drv.In(a), drv.In(b),drv.In(np.array([length],dtype=np.int32)), drv.Out(c), grid=(1,1,1), block=(length,1,1))

print(a,b,c,sep='\n')
print(f"差异：{c-a-b}")

[0.67766273 0.7226967  0.76874584 0.75571066 0.44818532]
[0.11860567 0.34655762 0.79528314 0.78939086 0.07895472]
[0.7962684 1.0692544 1.564029  1.5451015 0.52714  ]
差异：[ 7.4505806e-09  5.9604645e-08  0.0000000e+00  0.0000000e+00
 -2.2351742e-08]


# 规约运算